# MLX MinGPT


In [107]:
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
from mlx.utils import tree_flatten, tree_map, tree_unflatten
from bitlinear import BitLinear, test_bitlinear_forward_pass, test_bitlinear_initialization, test_bitlinear_no_bias, test_bitlinear_quantization

In [108]:
bitlinear = BitLinear(10, 6, bits=2)
input_tensor = mx.random.normal([6, 10])  # Example input tensor
output = bitlinear(input_tensor)
print(mx.round(output))  # Example output tensor
test_bitlinear_initialization()
test_bitlinear_forward_pass()
test_bitlinear_no_bias()
test_bitlinear_quantization()

array([[-0, -0, -0, -0, -1, -1],
       [0, 0, 1, 0, 0, 0],
       [1, -1, 0, 0, 0, -1],
       [1, -0, -0, -0, -0, 0],
       [0, -0, 0, 0, -0, 0],
       [1, 0, 0, 0, 0, 0]], dtype=float32)


In [109]:

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.00


In [110]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = mx.default_device()


In [111]:
# ------------

mx.random.seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [112]:

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


In [113]:

# Train and test splits
data = mx.array(encode(text), dtype=mx.int64)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


In [114]:

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = mx.random.randint(0, len(data) - block_size, (batch_size,))
    ix = [i.item() for i in ix]
    x = mx.stack([data[i:i+block_size] for i in ix])
    y = mx.stack([data[i+1:i+block_size+1] for i in ix])
    # x, y = x.to(device), y.to(device)
    return x, y


In [115]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = BitLinear(n_embd, head_size, bias=False)
        self.query = BitLinear(n_embd, head_size, bias=False)
        self.value = BitLinear(n_embd, head_size, bias=False)
        self.tril = mx.tril(mx.ones([block_size, block_size]))
        self.dropout = nn.Dropout(dropout)

    def __call__(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose((0,2,1)) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        mask = self.tril[:T, :T] == 0
        wei = mx.where(mask, float('-inf'), wei) # (B, T, T)
        wei = nn.softmax(wei, axis=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


In [116]:

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = [Head(head_size) for _ in range(num_heads)]
        self.proj = BitLinear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def __call__(self, x):
        out = mx.concatenate([h(x) for h in self.heads], axis=-1)
        out = self.dropout(self.proj(out))
        mx.eval(out)
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            BitLinear(n_embd, 4 * n_embd),
            nn.ReLU(),
            BitLinear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def __call__(self, x):
        return self.net(x)


In [117]:

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def __call__(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [118]:
# super simple bigram model
class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = BitLinear(n_embd, vocab_size)

    def __call__(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(mx.arange(T)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        mx.eval(logits)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = nn.losses.cross_entropy(logits, targets, reduction='mean')

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, _ = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = nn.softmax(logits, axis=-1) # (B, C)
            # sample from the distribution
            idx_next = mx.random.categorical(probs, axis=-1, num_samples=1)
            # append sampled index to the running sequence
            idx = mx.concatenate((idx, idx_next), axis=1) # (B, T+1)
            mx.eval(idx)
        return idx


In [119]:

model = LanguageModel()

In [120]:
params = model.parameters()


In [121]:
# print the number of parameters in the model
p = sum(v.size for _, v in tree_flatten(model.parameters())) / 10**6
print(f"Total parameters {p:.3f}M")

Total parameters 0.240M


In [122]:
# create an optimizer
optimizer = optim.AdamW(learning_rate)

In [123]:
def estimate_loss(model):
    out = {}
    for split in ['train', 'val']:
        losses = mx.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

def loss_fn(model, x, y):
    _, loss = model(x, y)
    return loss

def step(model, optimizer, inputs, targets):
    loss_and_grad_fn = nn.value_and_grad(model, loss_fn)
    loss, grads = loss_and_grad_fn(model, inputs, targets)
    mx.eval(loss)
    mx.eval(grads)
    optimizer.update(model, grads)
    return loss


In [124]:
import time
start = time.time()
model.train()
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train'].item():.4f}, val loss {losses['val'].item():.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    step(model, optimizer, xb, yb)

end = time.time()
print(f"Training complete in {end-start}s")

step 0: train loss 4.2018, val loss 4.2021
step 100: train loss 3.3438, val loss 3.3474
step 200: train loss 3.0236, val loss 3.0435
step 300: train loss 2.8527, val loss 2.8860


In [ ]:
print(model)
tree_flatten(model)

LanguageModel(
  (token_embedding_table): Embedding(65, 64)
  (position_embedding_table): Embedding(32, 64)
  (blocks): Sequential(
    (layers.0): Block(
      (sa): MultiHeadAttention(
        (heads.0): Head(
          (key): BitLinear(input_dims=64, output_dims=16, bias=False
            (norm): LayerNorm(64, eps=1e-05, affine=True)
          )
          (query): BitLinear(input_dims=64, output_dims=16, bias=False
            (norm): LayerNorm(64, eps=1e-05, affine=True)
          )
          (value): BitLinear(input_dims=64, output_dims=16, bias=False
            (norm): LayerNorm(64, eps=1e-05, affine=True)
          )
          (dropout): Dropout(p=0.0)
        )
        (heads.1): Head(
          (key): BitLinear(input_dims=64, output_dims=16, bias=False
            (norm): LayerNorm(64, eps=1e-05, affine=True)
          )
          (query): BitLinear(input_dims=64, output_dims=16, bias=False
            (norm): LayerNorm(64, eps=1e-05, affine=True)
          )
          (value

[('token_embedding_table.weight',
  array([[-0.174924, -0.256206, -0.00602648, ..., -0.486972, 0.0349107, -0.433979],
         [0.671945, 0.513543, -0.155299, ..., -0.232984, -0.178144, -0.304057],
         [-0.256707, 0.0360707, -0.0291384, ..., 0.3691, 0.00463329, 0.25679],
         ...,
         [-0.215785, -0.0437228, -0.119936, ..., -0.172752, 0.353932, -0.0747143],
         [-0.435995, -0.301742, -0.0949814, ..., 0.124442, -0.163173, 0.18864],
         [0.108434, 0.0180779, -0.146745, ..., -0.0957022, 0.221605, 0.0811022]], dtype=float32)),
 ('position_embedding_table.weight',
  array([[-0.0720482, 0.0529957, -0.237977, ..., -0.0643842, 0.0388052, -0.00518194],
         [-0.0405347, 0.140975, -0.05469, ..., -0.0265713, -0.0371366, -0.105592],
         [0.0305713, 0.10734, 0.0803439, ..., -0.0341943, -0.105753, 0.0609333],
         ...,
         [0.0312081, -0.116281, -0.0712877, ..., 0.0789341, -0.145155, 0.000583636],
         [-0.0653304, 0.101005, 0.0639263, ..., 0.0999906, -0

In [ ]:
def compress_weights(x):
    x = mx.round(x)
    return x.astype(mx.int8)

# model.apply(lambda x: x.astype(mx.float16))
def is_bitlinear(module, key, value):
    return isinstance(value, BitLinear)

# model.apply(compress_weights)
model.apply(map_fn=compress_weights, filter_fn=is_bitlinear)
    # def valid_parameter_filter(module, key, value):
    #     return isinstance(value, (dict, list, mx.array)) and not key.startswith("_")

print(model)
print(tree_flatten(model))

LanguageModel(
  (token_embedding_table): Embedding(65, 64)
  (position_embedding_table): Embedding(32, 64)
  (blocks): Sequential(
    (layers.0): Block(
      (sa): MultiHeadAttention(
        (heads.0): Head(
          (key): BitLinear(input_dims=64, output_dims=16, bias=False
            (norm): LayerNorm(64, eps=1e-05, affine=True)
          )
          (query): BitLinear(input_dims=64, output_dims=16, bias=False
            (norm): LayerNorm(64, eps=1e-05, affine=True)
          )
          (value): BitLinear(input_dims=64, output_dims=16, bias=False
            (norm): LayerNorm(64, eps=1e-05, affine=True)
          )
          (dropout): Dropout(p=0.0)
        )
        (heads.1): Head(
          (key): BitLinear(input_dims=64, output_dims=16, bias=False
            (norm): LayerNorm(64, eps=1e-05, affine=True)
          )
          (query): BitLinear(input_dims=64, output_dims=16, bias=False
            (norm): LayerNorm(64, eps=1e-05, affine=True)
          )
          (value

In [ ]:
# save weights
model.save_weights("./bitlinearshakespearebigramweights.npz")


In [ ]:
def generate(model, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
        # crop idx to the last block_size tokens
        idx_cond = idx[:, -block_size:]
        # get the predictions
        logits, _ = model(idx_cond)
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        # probs = nn.softmax(logits, axis=-1) # (B, C)
        probs = logits
        # sample from the distribution
        idx_next = mx.random.categorical(probs, axis=-1, num_samples=1)
        # append sampled index to the running sequence
        idx = mx.concatenate((idx, idx_next), axis=1) # (B, T+1)
        mx.eval(idx)
    return idx


In [106]:
# generate from the model
model.eval()
print("Generating..")
context = mx.zeros((1, 1), dtype=mx.int8)
#generated = model.generate(context, max_new_tokens=1000)[0].tolist()
generated = generate(model, context, max_new_tokens=500)[0].tolist()
print(decode(generated))



Generating..



INnteut, me hey y,
s thay!
XE:
sefony it thayoso s bustou jchatoce,


lo momils, MSnm ulourouck t thtre.
Peay bed. acog henous.
A:
NfolMag E:
l e thetheDboieroL!



APnthatheteAPl e 'leshoDUI
WithoyREI brfe, t, ase tiloyohe No,
lins se yf mouthedeD ven iveO;
soplk'l DinFilatead , are! wotheD
d hiY E is,
s.
AnRutFsete E hgoresthpanho prll palath,
Nebe sed herce, mfou w wodcove thorso themenMloomeamD:
yoUyo w uVIm. yo.
-hed otham Lu st joGk, pr asf terosfouOLKe dand. Lhtpriy.
CPatEt.
IILthis:



